<a href="https://colab.research.google.com/github/cheffjiu/pytorch-tutorials-zh/blob/main/data_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| **Datasets & DataLoaders** \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Datasets & DataLoaders
======================


处理数据样本的代码可能会变得杂乱无章且难以维护；理想情况下，为了更好的可读性和模块化，我们希望数据集代码与模型训练代码解耦。PyTorch 提供了两个数据原语：`torch.utils.data.DataLoader` 和 `torch.utils.data.Dataset`，它们允许你使用预加载的数据集以及你自己的数据。`Dataset` 存储样本及其相应的标签，而 `DataLoader` 则把`Dataset` 包装一个可迭代对象，以便轻松访问样本。
PyTorch领域库提供了许多预加载的数据集（如FashionMNIST），这些数据集继承自`torch.utils.data.Dataset`类，并实现了特定于特定数据的函数。它们可用于为你的模型制作原型并进行基准测试。你可以在这里找到它们。: [Image
Datasets](https://pytorch.org/vision/stable/datasets.html), [Text
Datasets](https://pytorch.org/text/stable/datasets.html), and [Audio
Datasets](https://pytorch.org/audio/stable/datasets.html)


Loading a Dataset（加载数据集）
=================

以下是一个如何从TorchVision加载[Fashion-MNIST](https://research.zalando.com/project/fashion_mnist/fashion_mnist/) ，Fashion-MNIST是Zalando公司的一个商品图片数据集，由60000个训练样本和10000个测试样本组成。每个样本包含一张28×28的灰度图像以及一个来自10个类别之一的关联标签。

我们使用以下参数加载[FashionMNIST数据集](https://pytorch.org/vision/stable/datasets.html#fashion-mnist):

   -   `root` 是存储训练/测试数据的路径,   
   -   `train` 指定训练或测试数据集，  
   -   `download=True` 如果数据在`root`路径下不可用，则从互联网下载数据。    
   -   `transform` and `target_transform` 指定特征和标签变换


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Iterating and Visualizing the Dataset（迭代和可视化数据集）
=====================================

我们可以像操作列表一样手动对`Datasets`进行索引：`training_data[index]`。我们使用`matplotlib`来可视化训练数据中的一些样本。.


In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

------------------------------------------------------------------------


Creating a Custom Dataset for your files（为你的文件创建自定义数据集）
========================================

自定义数据集类必须实现三个函数:
`__init__`, `__len__`, 和`__getitem__`. 看看这个实现：FashionMNIST图像存储在目录`img_dir`中，它们的标签则单独存储在CSV文件`annotations_file`中。.

在接下来的章节中，我们将详细分析这些函数中每一个的运行情况。


In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])#iloc 是 pandas 库中 DataFrame 对象的一个属性，用于通过整数位置（从 0 开始的索引）来选取行和列
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

`__init__`
==========

在实例化`Dataset`对象时，`__init__`函数会运行一次。我们会初始化包含图像的目录、注释文件以及两种变换（下一节将详细介绍）。 .

The labels.csv file looks like: :

    tshirt1.jpg, 0
    tshirt2.jpg, 0
    ......
    ankleboot999.jpg, 9


In [ ]:
def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
    self.img_labels = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

`__len__`
=========

`__len__` 函数返回我们数据集中样本的数量.

Example:


In [ ]:
def __len__(self):
    return len(self.img_labels)

`__getitem__`
=============

`__getitem__`函数根据给定的索引 `idx` 从数据集中加载并返回一个样本。它依据索引确定图像在磁盘上的位置，使用 `read_image` 将其转换为张量，从 `self.img_labels` 中的 csv 数据里获取相应的标签，对它们调用转换函数（如果适用），然后以元组形式返回张量图像和相应的标签。

In [ ]:
def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])#iloc 是 pandas 库中 DataFrame 对象的一个属性，用于通过整数位置（从 0 开始的索引）来选取行和列
    image = read_image(img_path)
    label = self.img_labels.iloc[idx, 1]
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

------------------------------------------------------------------------


Preparing your data for training with DataLoaders(使用数据加载器为训练准备数据)
=================================================

`Dataset` 一次获取我们数据集中一个样本的特征和标签。在训练模型时，我们通常希望以 “小批量” 方式传递样本，在每个时期重新打乱数据以减少模型过拟合，并使用 Python 的 `multiprocessing` 来加速数据获取。

`DataLoader` 是一个可迭代对象，它通过简单的API为我们抽象了这种复杂性。.


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

Iterate through the DataLoader(遍历数据加载器)
==============================

我们已经将该数据集加载到`DataLoader`中，并可以根据需要遍历该数据集。下面的每次迭代都会返回一批`train_features`和`train_labels`（分别包含`batch_size=64`个特征和标签）。由于我们指定了`shuffle=True`，在遍历完所有批次后，数据会被打乱（如需对数据加载顺序进行更精细的控制，请查看
[Samplers](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler)).


In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

------------------------------------------------------------------------


Further Reading
===============

-   [torch.utils.data API](https://pytorch.org/docs/stable/data.html)
